# LDAについてのまとめ

LDAの手順や性格についての理解を深めるため、事例をものすごく簡素化してみました。

## 事前の仕込み

あとで使用するユーティリティー関数です。すみませんがここは読み飛ばしてください。。。

In [1]:
def get_top_words_in_topic(lda, tf_feature_names, n_top_words):
    top_words = []

    for topic_idx, topic in enumerate(lda.components_):
        # 特徴語のインデックスを、単語出現確率の高いもの順に取得
        top_words_idx_list = topic.argsort()
        reverse_idx = -n_top_words - 1
        top_words_idx = top_words_idx_list[:reverse_idx:-1]

        # 特徴語をリストに取得
        top_words_list = [tf_feature_names[i] for i in top_words_idx]
        top_words.append(top_words_list)

    # 上位の特徴語リストを戻す
    return top_words

def print_top_words_in_topic(lda, tf_feature_names, n_top_words):
    top_words_list = get_top_words_in_topic(lda, tf_feature_names, n_top_words)
    for topic_idx, top_words in enumerate(top_words_list):
        print("Topic #%d: %s" % (topic_idx, ",".join(top_words)))

def get_hit_topic_idx(probs):
    topic_idx = 0
    max_prob = 0.0
    for i in range(0, len(probs)):
        if max_prob < probs[i]:
            topic_idx = i
            max_prob = probs[i]

    # 質問文が属するトピックのインデックスを戻す
    return topic_idx

def print_text_and_topic_idx(corpus, answers):
    for data_idx, data in enumerate(corpus):
        topic_idx = get_hit_topic_idx(answers[data_idx])
        topic_prob_rate = answers[data_idx][topic_idx] * 100

        print("Question Text #%d ---> Topic #%d (%0.1f%%) (contents=[%s])" % (data_idx, topic_idx, topic_prob_rate, data))

## 初回学習

テストデータは以下の通りです。

そもそも LDA は教師なし学習のため、初回学習の段階では、データがカテゴライズされていない状態を想定しているようです。

### 学習データの仕込み

In [2]:
# 本当は事前にデータが分類されていることは想定されていないのですが。。。
# 簡単にするために、３分類のテストデータを使います。
corpus = []

# 質問分類１
corpus.append('Do you like an apple ?')
corpus.append('Do you like an orange ?')
corpus.append('Do you like a lemon ?')


# 質問分類２
corpus.append('Does he have a note ?')
corpus.append('Does she have a paper ?')
corpus.append('Does he have a watch ?')


# 質問分類３
corpus.append('Did she get a computer ?')
corpus.append('Did you get a camera ?')
corpus.append('Did he get a car ?')


corpus

['Do you like an apple ?',
 'Do you like an orange ?',
 'Do you like a lemon ?',
 'Does he have a note ?',
 'Does she have a paper ?',
 'Does he have a watch ?',
 'Did she get a computer ?',
 'Did you get a camera ?',
 'Did he get a car ?']

### 初回学習のインプットとなる、TFベクターを作成

事例を簡単にするため、インプットとなるTFベクターは、手作業で作成しております。

In [3]:
tf_feature_names = []
tf_feature_names.append('like')
tf_feature_names.append('have')
tf_feature_names.append('get')
tf_feature_names # これは特徴語のリストです。

['like', 'have', 'get']

In [4]:
from numpy import array
from scipy.sparse.csr import csr_matrix
tf_vector_array = array([
    [1, 0, 0],
    [1, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 1, 0],
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 1],
    [0, 0, 1]
    ])
tf_vector = csr_matrix(tf_vector_array)

tf_vector.toarray() # これは特徴語の出現回数リストです。

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=int64)

### 初回学習実行

In [5]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=3, # トピック分類件数
                                max_iter=10, # 繰り返し実行
                                learning_method='online', # 学習実行-->即結果を出力
                                learning_offset=50.,
                                random_state=1)
lda.fit(tf_vector)
answers = lda.transform(tf_vector)

### 学習の結果、分類されたトピック

質問分類１（like --> Topic #2）、質問分類２（have --> Topic #1）、質問分類３（bring --> Topic #0）におおむね分かれていると言えます。

ただし、分類されるべきトピックや、トピックが含まれる特徴語は、学習時にあらかじめ与えることができないので、トピック分類の順番などは当然ランダムになってしまうかと思われます。

In [6]:
n_top_words = 5
print_top_words_in_topic(lda, tf_feature_names, n_top_words)

Topic #0: like,get,have
Topic #1: have,get,like
Topic #2: get,like,have


### それぞれの質問文が、どのトピックに属しているかを確認

In [7]:
# 全質問文について検証してみます。。。
print_text_and_topic_idx(corpus, answers)

Question Text #0 ---> Topic #0 (65.5%) (contents=[Do you like an apple ?])
Question Text #1 ---> Topic #0 (65.5%) (contents=[Do you like an orange ?])
Question Text #2 ---> Topic #0 (65.5%) (contents=[Do you like a lemon ?])
Question Text #3 ---> Topic #1 (65.8%) (contents=[Does he have a note ?])
Question Text #4 ---> Topic #1 (65.8%) (contents=[Does she have a paper ?])
Question Text #5 ---> Topic #1 (65.8%) (contents=[Does he have a watch ?])
Question Text #6 ---> Topic #2 (65.7%) (contents=[Did she get a computer ?])
Question Text #7 ---> Topic #2 (65.7%) (contents=[Did you get a camera ?])
Question Text #8 ---> Topic #2 (65.7%) (contents=[Did he get a car ?])


## ２回め以降の学習

### テストデータ、およびTFベクターの作成

あたえるテストデータは、初回学習時と、TFベクターの次元数、特徴語のインデックスが完全一致している必要があります。

これは、LDAの学習時に、TFベクターしか入力できないための制約になります。

In [8]:
corpus_add = []
corpus_add.append('Do you like a fish ?') # 質問分類１
corpus_add

['Do you like a fish ?']

In [9]:
array1 = array([
    [1, 0, 0]
    ])
tf_vector_add = csr_matrix(array1)
tf_vector_add.toarray() # これは特徴語の出現回数リストです。

array([[1, 0, 0]], dtype=int64)

### 追加学習を実行

In [10]:
lda.partial_fit(tf_vector_add)
answers_add = lda.transform(tf_vector_add)

分類結果はというと・・・正しく分類されているようです

In [11]:
# 全質問文について検証してみます。。。
print_text_and_topic_idx(corpus_add, answers_add)

Question Text #0 ---> Topic #0 (33.4%) (contents=[Do you like a fish ?])


### さらに追加学習を実行

In [12]:
corpus_add2 = []
corpus_add2.append('Do you like a fish ?') # 質問分類１
corpus_add2.append('Does he have a money ?') # 質問分類２
corpus_add2.append('Do you like a frog ?') # 質問分類１
corpus_add2.append('Did you get a bike ?') # 質問分類３
corpus_add2

['Do you like a fish ?',
 'Does he have a money ?',
 'Do you like a frog ?',
 'Did you get a bike ?']

In [13]:
array2 = array([
    [1, 0, 0],
    [0, 1, 0],
    [1, 0, 0],
    [0, 0, 1]
    ])
tf_vector_add2 = csr_matrix(array2)
tf_vector_add2.toarray() # これは特徴語の出現回数リストです。

array([[1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1]], dtype=int64)

In [14]:
lda.partial_fit(tf_vector_add2)
answers_add2 = lda.transform(tf_vector_add2)

In [15]:
# 全質問文について検証してみます。。。
print_text_and_topic_idx(corpus_add2, answers_add2)

Question Text #0 ---> Topic #0 (63.8%) (contents=[Do you like a fish ?])
Question Text #1 ---> Topic #1 (66.7%) (contents=[Does he have a money ?])
Question Text #2 ---> Topic #0 (63.8%) (contents=[Do you like a frog ?])
Question Text #3 ---> Topic #2 (66.6%) (contents=[Did you get a bike ?])
